In [1]:
# ======================== IMPORTS ============================
import os, json, faiss, numpy as np, pytesseract, PyPDF2, threading, queue, pyttsx3
import speech_recognition as sr
import gradio as gr
from PIL import Image
from sentence_transformers import SentenceTransformer
import ollama
from duckduckgo_search import DDGS
from datetime import datetime
import random

# Store dynamic assignment per prompt
current_arena_assignment = {}

# ======================== CHATBOT CLASS ============================
class ChatBot:
    def __init__(self, model="llama3.2:3b"): # Can use llama3.2:3b or llama3.2:1b  or llama3-1b-spamgen 
        self.model = model
        self.system_prompt = "Answer by clear small useful answers. Short responses."
        self.global_settings = {"max_tokens": 50, "temperature": 0.7}
        self.chat_history = [{'role': 'system', 'content': self.system_prompt}]

        self.rag_embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.rag_chunks, self.rag_index = [], None

        self.voice_enabled = False
        self.recognizer = sr.Recognizer()
        self.mic = sr.Microphone()
        self.tts_queue = queue.Queue()
        threading.Thread(target=self._tts_loop, daemon=True).start()

    # ----------- Text-to-Speech ------------
    def _tts_loop(self):
        while True:
            text = self.tts_queue.get()
            if text is None:
                break
            try:
                engine = pyttsx3.init()  # move inside the loop
                engine.setProperty('voice', 'HKEY_LOCAL_MACHINE\\SOFTWARE\\Microsoft\\Speech\\Voices\\Tokens\\TTS_MS_EN-US_ZIRA_11.0')
                engine.say(text)
                engine.runAndWait()
                engine.stop()
            except Exception as e:
                print(f"TTS error: {e}")


    def speak(self, text):
        if self.voice_enabled:
            self.tts_queue.put(text)

    def stop_audio(self):
        self.tts_queue.put(None)
        self.voice_enabled = False

    # ----------- Document Loading ------------
    def load_document(self, file_objs):
        chunks, summaries = [], []
        for f in file_objs:
            try:
                reader = PyPDF2.PdfReader(f.name)
                texts = [p.extract_text().strip() for p in reader.pages if p.extract_text()]
                chunks.extend(texts)
                summaries.append(f"**{os.path.basename(f.name)}**: " + "\n".join(texts[:2]))
            except Exception as e:
                summaries.append(f"Error reading {f.name}: {e}")
        if chunks:
            self.rag_chunks = chunks
            embeddings = self.rag_embedder.encode(chunks)
            self.rag_index = faiss.IndexFlatL2(embeddings.shape[1])
            self.rag_index.add(np.array(embeddings))
            return "\n\n".join(summaries)
        return "No valid PDF content loaded."
    # ----------- OCR Image Loading ------------

    def load_image(self, file_obj):
        try:
            text = pytesseract.image_to_string(Image.open(file_obj.name)).strip()
            if text:
                self.rag_chunks = [text]
                embeddings = self.rag_embedder.encode(self.rag_chunks)
                self.rag_index = faiss.IndexFlatL2(embeddings.shape[1])
                self.rag_index.add(np.array(embeddings))
                return f"Extracted text:\n{text[:500]}..."
            return "No text found."
        except Exception as e:
            return f"Image error: {e}"

    # ----------- Context Retrieval ------------
    def retrieve_context(self, query, top_k=2):
        if not self.rag_index: return ""
        query_emb = self.rag_embedder.encode([query])
        D, I = self.rag_index.search(np.array(query_emb), top_k)
        return "\n".join(self.rag_chunks[i] for i in I[0])

    # ----------- Internet Search ------------
    def internet_search(self, query, max_results=3):
        results = []
        try:
            with DDGS() as ddgs:
                for r in ddgs.text(query, max_results=max_results):
                    results.append(f"{r.get('title', '')}: {r.get('body', '')} ({r.get('href', '')})")
        except Exception as e:
            results.append(f"Search error: {e}")
        return "\n".join(results)

    # ----------- Chat Functions ------------
    def chat(self, user_input, search_enabled=False):
        search_context = self.internet_search(user_input) if search_enabled else ""
        context = ""
        if search_enabled:
            context += "Relevant internet search results:\n" + self.internet_search(user_input) + "\n"
        rag_context = self.retrieve_context(user_input)
        if rag_context:
            context += "RAG user document:\n" + rag_context + "\n"
        prompt = f"Context:\n{context}\nQuestion: {user_input}\nAnswer:" if context else user_input

        self.chat_history.append({'role': 'user', 'content': prompt})
        response = ollama.chat(model=self.model, messages=self.chat_history, options=self.global_settings)
        reply = response['message']['content'].strip()
        self.chat_history.append({'role': 'assistant', 'content': reply})
        self.speak(reply)
        return reply, search_context
    # ----------- Voice Input ------------
    def listen(self):
        with self.mic as source:
            audio = self.recognizer.listen(source)
        try:
            return self.recognizer.recognize_google(audio)
        except:
            return "Voice recognition error."


c:\Users\33766\Documents\Travail\TU Delft\Q3\Machine Learning Bayesian\Assignment2ML\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ======================== GRADIO INTERFACE (NO ARENA MODE) ============================

bot = ChatBot()  # Initialize chatbot instance

# ---------------- Helper Functions ----------------

def handle_upload(files):
    if not files:
        return "No file uploaded."
    file = files[0]
    return bot.load_document(files) if file.name.lower().endswith("pdf") else bot.load_image(file)

def toggle_voice_output():
    bot.voice_enabled = not bot.voice_enabled
    return "🔊" if bot.voice_enabled else "🔇"

def clear_chat():
    bot.chat_history = [{'role': 'system', 'content': bot.system_prompt}]
    return []

def toggle_search(search_state):
    new_state = not search_state
    return new_state, f"Search: {'ON' if new_state else 'OFF'}"

# ---------------- Chat Handler ----------------
def handle_main(msg, history, search):
    reply, sctx = bot.chat(msg, search_enabled=search)
    history.append((msg, reply))
    return history, "", sctx

# ---------------- Gradio Interface ----------------

with gr.Blocks(css=".gradio-container {width:100%; max-width:none;}") as demo:
    gr.Markdown("# Delftbot 🔥")

    # States to persist toggle buttons
    search_state = gr.State(False)

    # Chat display and input textbox
    chat_display = gr.Chatbot(label="Chat History")
    user_input = gr.Textbox(label="Your Message", placeholder="Type here...")

    # Control buttons
    with gr.Row():
        mic_btn = gr.Button("🎤")
        speaker_btn = gr.Button("🔇")
        search_btn = gr.Button("Search: OFF")
        clear_btn = gr.Button("🧹 Clear Chat")

    # File upload and summaries
    file_upload = gr.File(label="Upload PDF/Image", file_count="multiple")
    file_summary = gr.Textbox(label="File Summary", interactive=False)

    # ---------------- Event Bindings ----------------

    # Submit text input
    user_input.submit(fn=handle_main,
                      inputs=[user_input, chat_display, search_state],
                      outputs=[chat_display, user_input, file_summary])

    # Submit via mic input (speech-to-text)
    mic_btn.click(fn=lambda h, s: handle_main(bot.listen(), h, s),
                  inputs=[chat_display, search_state],
                  outputs=[chat_display, user_input, file_summary])

    # Toggle TTS audio
    speaker_btn.click(fn=toggle_voice_output, outputs=speaker_btn)

    # Toggle search mode
    search_btn.click(fn=toggle_search,
                     inputs=[search_state],
                     outputs=[search_state, search_btn])

    # Clear chat
    clear_btn.click(fn=clear_chat, outputs=chat_display)

    # Handle file upload (PDF/Image)
    file_upload.change(fn=handle_upload, inputs=file_upload, outputs=file_summary)

    # Enable request queuing and launch interface
    demo.queue()
    demo.launch()


C:\Users\33766\AppData\Local\Temp\ipykernel_30016\2178421024.py:40: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_display = gr.Chatbot(label="Chat History")


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
